# Locking algorithms

Programs with multiple threads or processes running at the same time have a downside when it comes to the use of shared datastructures: You have to prevent that two running opponent threads or processes write at the same time to the data structure. Depending on the data structure itself it might be left in an unconsistent state or data is lost. Therefore, methods to prevent rare events that cause these issues are important to know.

Let's examine an example: Imagine two people are booking the last flight on an airlilne. Bot people don't know that this is the last flight, they just visited the booking website, registered for a ticket and now want to pay their order. Person A clicked first on the payment button but takes a lot of time to enter their credit card information. While person A is typing, person B clicks the same button on another computer, enters the credit card information faster thgan person A and leaves the website with a ticket. Seconds after that person A is also ready and recieves the same ticket, thus resulting in a single ticket being sold to two persons.

Let's introduce a simple lock to this mechanism: When the ordering process starts, the website first checks if there is a lock on the ticket and if this ist't the case, it sets the lock, proceeding like the ticket belongs to this buyer. This works in most cases but under the rare occasion when person A and person B both click the button at the same time, both of their website instances register that no lock is set so they set their own value. If the lock was just a variable, then only one of the writing operations on that variable would be winning this so calles "race condition" and the lock would be looking consistent, but both buyers can proceed with their checkout and we have the same issue than in the previous example.

This is the reason why a sophisticated locking mechanis is needed to prevent race conditions. In order to operate well, we have to follow these rules:

1. No two processes can be simultaneously in the same critical region
2. You cannot make any assumption about the number of processes on a machine running in parallel as well as the speed of these processes
3. No process outside its critical region should block any other process
4. No process should wait forever to enter its critical region

Let's take a look on some locking mechanisms

## Prerequisites

These are the base packages to create shared memory regions and spawn processes

In [1]:
# multiprocessing module to spawn processes
import multiprocessing as mp

# shared memory between the processes
from multiprocessing import shared_memory as shm

# delays
import time
from random import randint

def random_delay():
    time.sleep(randint(0, 1000) / 1000)

## Peterson Algorithm

Also known as "the canadian processes"

In [2]:
# first create a shared buffer between the processes
share = shm.SharedMemory(create=True, size=4) # size of 4 byted
# reset all tje values in the buffer to 0
share.buf[:4] = bytearray([0,0,0,0])

# get the information to attach to the shared buffer later on
share_name = share.name

# now define the kind of process that wants to access the buffer and write the shared variable:
def process(n: int, share_name: str):
    share = shm.SharedMemory(share_name)  # attach to the shared memory

    # the parameter n is the number of the process: 0 or 1
    other = 1 - n  # get name of the other process

    def enter_region():
        # begin entering critical region
        # first notify the other process that you are interested in the critical region
        share.buf[1 + n] = True
        # you are the last one who wants to enter the critical region so set yourself to the "last" position:
        share.buf[3] = n
        # busy waiting
        while (share.buf[1 + other] == True) and (share.buf[3] == n):
            pass
        # end entering critical region
        print(f"Process {n} entered the critical region")

    def leave_region():
        # leave critical region
        print(f"Process {n} left the critical region")
        share.buf[1 + n] = False

    for i in range(5):  # increment the shared variable 5 times
        enter_region()
        
        # now do something
        last_state = share.buf[0]
        share.buf[0] += 1
        print(f"Process {n} changed the shared variable from {last_state} to {share.buf[0]}")
        # wait a little bit
        random_delay()
        # something was done :)
        
        leave_region()

        # wait again to simulate workload
        random_delay()
        
    # release the buffer before returning
    share.close()  

# now define the two processes that are competing against each other
processes = [mp.Process(target=process, args=(n, share_name,)) for n in range(2)]
# start all processes
for i in processes:
    i.start()
# wait till all processes end
for i in processes:
    i.join()

# remove shared memory
share.unlink()

Process 0 entered the critical region
Process 0 changed the shared variable from 0 to 1
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 1 to 2
Process 1 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 2 to 3
Process 1 left the critical region
Process 0 entered the critical region
Process 0 changed the shared variable from 3 to 4
Process 0 left the critical region
Process 0 entered the critical region
Process 0 changed the shared variable from 4 to 5
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 5 to 6
Process 1 left the critical region
Process 0 entered the critical region
Process 0 changed the shared variable from 6 to 7
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 7 to 8
Process 1 left the critical region
Process 0 entere

## Generalized Peterson Algorithm

For N >= 2

In [3]:
# number of parallel processes
N = 4

# shared variable that all processes want to access
var = shm.SharedMemory(create=True, size=1)  # 1 byte integer
var.buf[0] = 0

# an array to indicate the current level for each process
level = shm.SharedMemory(create=True, size=N)  # N integer values
level.buf[:] = bytearray([0] * N)  # reset all values to 0

# the information, which process is the last one who entered a level
last = shm.SharedMemory(create=True, size=N)
last.buf[:] = bytearray([0] * N)

def process(n: int, var: str, level: str, last: str):
    # get access to shared memory regions
    var = shm.SharedMemory(var)
    level = shm.SharedMemory(level)
    last = shm.SharedMemory(last)

    def enter_region():
        for k in range(1, N):
            level.buf[n] = k
            last.buf[k] = n
            # now wait until nobody else is in the same ring
            for j in range(N):
                if j == n:
                    continue
                while level.buf[j] >= k and last.buf[k] == n:
                    pass
        print(f"Process {n} entered the critical region")

    def leave_region():
        print(f"Process {n} left the critical region")
        level.buf[n] = 0

    for i in range(5):  # increment the shared variable 5 times
        enter_region()
        
        # now do something
        last_state = var.buf[0]
        var.buf[0] += 1
        print(f"Process {n} changed the shared variable from {last_state} to {var.buf[0]}")
        # wait a little bit
        random_delay()
        # something was done :)
        
        leave_region()

        # wait again to simulate workload
        random_delay()           

    # detach access to shared memories
    var.close()
    level.close()
    last.close()

# now define the two processes that are competing against each other
processes = [mp.Process(target=process, args=(n, var.name, level.name, last.name,)) for n in range(N)]
# start all processes
for i in processes:
    i.start()
# wait till all processes end
for i in processes:
    i.join()

# remove shared memories
var.unlink()
level.unlink()
last.unlink()

Process 0 entered the critical region
Process 0 changed the shared variable from 0 to 1
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 1 to 2
Process 1 left the critical region
Process 2 entered the critical region
Process 2 changed the shared variable from 2 to 3
Process 2 left the critical region
Process 3 entered the critical region
Process 3 changed the shared variable from 3 to 4
Process 3 left the critical region
Process 0 entered the critical region
Process 0 changed the shared variable from 4 to 5
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 5 to 6
Process 1 left the critical region
Process 2 entered the critical region
Process 2 changed the shared variable from 6 to 7
Process 2 left the critical region
Process 3 entered the critical region
Process 3 changed the shared variable from 7 to 8
Process 3 left the critical region
Process 0 entere

## Single control thread

Use a multiprocessing queue to pass an "access token" to all the threads. The access token can be anything, as long as there is only one token available there is only one process accessing it. Python wraps a mutex around the queue so that only one process at the time has read / write access to the queue. If the queue is empty, the process blocks. 

In [5]:
# number of parallel processes
N = 4

# shared variable that all processes want to access
var = shm.SharedMemory(create=True, size=1)  # 1 byte integer
var.buf[0] = 0

# create a queue to pass the "read token between all the processes
queue = mp.Queue(N)

def process(n: int, var: str, queue: str):
    # get access to shared memory regions
    var = shm.SharedMemory(var)

    def enter_region():
        _ = queue.get(block=True)  # wait for the read token to be in the queue
        print(f"Process {n} entered the critical region")

    def leave_region():
        print(f"Process {n} left the critical region")
        queue.put(True)

    for i in range(5):  # increment the shared variable 5 times
        enter_region()
        
        # now do something
        last_state = var.buf[0]
        var.buf[0] += 1
        print(f"Process {n} changed the shared variable from {last_state} to {var.buf[0]}")
        # wait a little bit
        random_delay()
        # something was done :)
        
        leave_region()
        # wait again to simulate workload
        random_delay()           

    # detach access to shared memories
    var.close()


# now define the two processes that are competing against each other
processes = [mp.Process(target=process, args=(n, var.name, queue,)) for n in range(N)]
# enter the first access token into the queue
queue.put(True)  # use a bookean true as the access token, the token can also be anything else
# start all processes
for i in processes:
    i.start()
# wait till all processes end
for i in processes:
    i.join()

# clear the queue
queue.close()
# remove shared memories
var.unlink()

Process 0 entered the critical region
Process 0 changed the shared variable from 0 to 1
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 1 to 2
Process 1 left the critical region
Process 2 entered the critical region
Process 2 changed the shared variable from 2 to 3
Process 2 left the critical region
Process 3 entered the critical region
Process 3 changed the shared variable from 3 to 4
Process 3 left the critical region
Process 0 entered the critical region
Process 0 changed the shared variable from 4 to 5
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 5 to 6
Process 1 left the critical region
Process 2 entered the critical region
Process 2 changed the shared variable from 6 to 7
Process 2 left the critical region
Process 3 entered the critical region
Process 3 changed the shared variable from 7 to 8
Process 3 left the critical region
Process 0 entere

## Mutex

The python `lock` mimics a mutex. It uses atomic operations (TSL / XCHG)

In [7]:
# number of parallel processes
N = 4

# shared variable that all processes want to access
var = shm.SharedMemory(create=True, size=1)  # 1 byte integer
var.buf[0] = 0

# the lock is a mutex
lock = mp.Lock()

def process(n: int, var: str, lock: str):
    # get access to shared memory regions
    var = shm.SharedMemory(var)

    def enter_region():
        lock.acquire()
        print(f"Process {n} entered the critical region")

    def leave_region():
        print(f"Process {n} left the critical region")
        lock.release()

    for i in range(5):  # increment the shared variable 5 times
        enter_region()
        
        # now do something
        last_state = var.buf[0]
        var.buf[0] += 1
        print(f"Process {n} changed the shared variable from {last_state} to {var.buf[0]}")
        # wait a little bit
        random_delay()
        # something was done :)
        
        leave_region()
        # wait again to simulate workload
        random_delay()           

    # detach access to shared memories
    var.close()


# now define the two processes that are competing against each other
processes = [mp.Process(target=process, args=(n, var.name, lock,)) for n in range(N)]
# start all processes
for i in processes:
    i.start()
# wait till all processes end
for i in processes:
    i.join()

# remove lock
del lock
# remove shared memories
var.unlink()

Process 0 entered the critical region
Process 0 changed the shared variable from 0 to 1
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 1 to 2
Process 1 left the critical region
Process 2 entered the critical region
Process 2 changed the shared variable from 2 to 3
Process 2 left the critical region
Process 3 entered the critical region
Process 3 changed the shared variable from 3 to 4
Process 3 left the critical region
Process 0 entered the critical region
Process 0 changed the shared variable from 4 to 5
Process 0 left the critical region
Process 1 entered the critical region
Process 1 changed the shared variable from 5 to 6
Process 1 left the critical region
Process 2 entered the critical region
Process 2 changed the shared variable from 6 to 7
Process 2 left the critical region
Process 0 entered the critical region
Process 0 changed the shared variable from 7 to 8
Process 0 left the critical region
Process 3 entere

## Semaphore

A mutex (aka. lock) is just a binary semaphore. There is not mutch to say about the semaphore because it works the same way as the lock. The big difference is that more than one process can acquire the access the critical section at the same time. The number of "access tokens" can be specified during the initialisation with the default being `1`.

Just use `mp.Semaphore()` instead of the lock in the initialisation process.

Tbh. it's hard to find a practical example for a semaphore.